In [2]:
!pip install scikit-learn


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 1.9 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/11.1 MB 2.0 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.1 MB 2.1 MB/s eta 0:00:05
   ------ --------------------------------- 1.8/11.1 MB 1.8 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/11.1 MB 1.9 MB/s eta 0:00:05
   --------- ------------------------------ 2.6/11.1 MB 1.9 MB/s eta 0:00:05
   ------------ --------------------------- 3.4/11.1 MB 2.1 MB/s eta 0:00:04
   ------------- -------------------------- 3.7/11.1 MB 2.1 MB/s eta 0:00:04
   --------------- ------------------------ 4.2/11.1 MB 2.1 MB/s eta 0:00:04
   ---------------- ----------------------- 4.5/11.1 MB 2.1 MB/s eta 0:00:04
   ----------------- ---------------------- 5.0/11.1 MB 2.1 MB/s eta 0:00:03
   ----------

In [ ]:
customers = pd.read_csv(r"C:\Users\janac\Downloads\Customers.csv")
transactions = pd.read_csv(r"C:\Users\janac\Downloads\transactions.csv")

In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
customers = pd.read_csv(r"C:\Users\janac\Downloads\Customers.csv")
transactions = pd.read_csv(r"C:\Users\janac\Downloads\transactions.csv")
products = pd.read_csv(r"C:\Users\janac\Downloads\products.csv" )

# Merge datasets for analysis
data = transactions.merge(customers, on="CustomerID").merge(products, on="ProductID")

# Aggregate customer-level features
# Aggregating customer-level features with the correct columns
customer_features = data.groupby("CustomerID").agg({
    "TotalValue": "sum",                # Total spending of the customer
    "Quantity": "sum",                  # Total quantity purchased
    "Price_y": "mean",                  # Average product price from Products.csv
    "Category": lambda x: x.mode()[0]   # Most frequent product category
}).reset_index()

# Renaming the column for clarity
customer_features.rename(columns={"Price_y": "AvgPrice"}, inplace=True)

# One-hot encoding the 'Category' column
customer_features = pd.get_dummies(customer_features, columns=["Category"], drop_first=True)

# Normalizing numerical features using StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features.drop("CustomerID", axis=1))

# Computing pairwise cosine similarity
similarity_matrix = cosine_similarity(scaled_features)

# Generating Lookalike Recommendations for the first 20 customers (C0001 to C0020)
lookalikes = {}

for i, customer_id in enumerate(customer_features["CustomerID"][:20]):
    # Get similarity scores for the current customer
    similar_scores = list(enumerate(similarity_matrix[i]))
    # Exclude self-similarity and sort by similarity score in descending order
    similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)[1:4]
    # Store the top 3 recommendations with similarity scores
    lookalikes[customer_id] = [(customer_features["CustomerID"][idx], round(score, 4)) for idx, score in similar_scores]

# Creating a DataFrame for Lookalike Recommendations
lookalike_df = pd.DataFrame({
    "CustomerID": lookalikes.keys(),
    "Recommendations": lookalikes.values()
})

# Saving the recommendations to a CSV file
lookalike_csv_path =r"C:\Users\janac\OneDrive\Desktop\kushala_Asanapuram_Lookalike.csv"
lookalike_df.to_csv(lookalike_csv_path, index=False)

print("Lookalike recommendations saved at:", lookalike_csv_path)


Lookalike recommendations saved at: C:\Users\janac\OneDrive\Desktop\kushala_Asanapuram_Lookalike.csv


In [4]:
import numpy as np

# Compute average intra-group similarity
def compute_intra_group_similarity(lookalikes, similarity_matrix, customer_features):
    intra_similarities = []
    for customer_id, recommendations in lookalikes.items():
        customer_idx = customer_features[customer_features["CustomerID"] == customer_id].index[0]
        for similar_id, _ in recommendations:
            similar_idx = customer_features[customer_features["CustomerID"] == similar_id].index[0]
            intra_similarities.append(similarity_matrix[customer_idx, similar_idx])
    return np.mean(intra_similarities)

# Compute inter-group dissimilarity 
def compute_inter_group_dissimilarity(similarity_matrix):
    return 1 - np.mean(similarity_matrix[np.triu_indices(similarity_matrix.shape[0], k=1)])

# Calculation of coverage
def compute_coverage(lookalikes, total_customers):
    return len(lookalikes) / total_customers

# Metrics calculation
intra_similarity = compute_intra_group_similarity(lookalikes, similarity_matrix, customer_features)
inter_dissimilarity = compute_inter_group_dissimilarity(similarity_matrix)
coverage = compute_coverage(lookalikes, len(customer_features))

# Print the results
print(f"Average Intra-group Similarity: {intra_similarity:.4f}")
print(f"Inter-group Dissimilarity: {inter_dissimilarity:.4f}")
print(f"Recommendation Coverage: {coverage:.2%}")


Average Intra-group Similarity: 0.9746
Inter-group Dissimilarity: 0.9982
Recommendation Coverage: 10.05%
